In [1]:
from google.genai import types
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.models.google_llm import Gemini
from google.adk.memory import InMemoryMemoryService
from google.adk.sessions import InMemorySessionService
from google.adk.tools import load_memory, preload_memory
from google.adk.agents.callback_context import CallbackContext

In [2]:
retry_config=types.HttpRetryOptions(attempts=3, exp_base=5, initial_delay=1, http_status_codes=[429, 500, 503, 504])
model = Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config)

In [ ]:
async def auto_save_to_memory(callback_context: CallbackContext):
    ctx = callback_context._invocation_context
    await ctx.memory_service.add_session_to_memory(ctx.session)

In [4]:
assistant_agent = Agent(
    name="AssistantAgent",
    model=model,
    instruction="You are a personal assistant",
    tools=[preload_memory], # or load_memory
    after_agent_callback=auto_save_to_memory,
)

In [5]:
session_service = InMemorySessionService()
memory_service = InMemoryMemoryService()

In [6]:
runner = Runner(agent=assistant_agent, app_name="Runner", session_service=session_service, memory_service=memory_service)

In [7]:
_ = await runner.run_debug("I love eating ice cream. What are some other desserts I might like?", session_id="session_1")


 ### Created new session: session_1

User > I love eating ice cream. What are some other desserts I might like?
AssistantAgent > That's great! Since you love ice cream, you probably enjoy desserts that are:

*   **Cold and refreshing:** This is a big one!
*   **Sweet:** Obviously!
*   **Creamy or smooth:** Many ice creams have a rich, creamy texture.
*   **Flavorful:** Whether it's fruity, chocolatey, or something else, you like a distinct taste.

Based on those preferences, here are some other desserts you might really enjoy:

**Similar Textures & Temperatures:**

*   **Frozen Yogurt (FroYo):** Very similar to ice cream but often a bit tangier and sometimes perceived as a healthier option.
*   **Sorbet/Sherbet:** If you like fruit flavors in your ice cream, you'll likely love sorbet. It's dairy-free and water-based, making it intensely refreshing and often very fruity. Sherbet is similar but usually contains a small amount of dairy or egg whites, giving it a slightly creamier texture

In [8]:
_ = await runner.run_debug("What do you know about my dessert preferences?", session_id="session_2")


 ### Created new session: session_2

User > What do you know about my dessert preferences?
AssistantAgent > Based on our previous conversation, I know that you love eating ice cream!

This tells me you likely enjoy desserts that are:

*   **Cold and refreshing**
*   **Sweet**
*   **Creamy or smooth in texture**
*   **Flavorful**

I've also suggested a number of desserts that share these qualities, such as frozen yogurt, sorbet, gelato, puddings, custards, cheesecakes, and mousse.

To help me refine my suggestions further, I asked you about your favorite ice cream flavors, your preferred level of sweetness, and any ingredients you dislike.
